For ease of running multiple experiments, the code has been extracted out of the python script and put into a jupyter notebook

Below are all settins for running experiments, please uncomment the four arguments corresponding with a certain experiment.

Make sure to run all cells. The last two cells will output the commandline scripts to run the ranklib script, and then run the evaluation afterwards.


In [ ]:
import gensim
import pandas as pd
from scipy import spatial
import numpy as np
import json
import os
class args(object):

    embedding = '../src/WKN-vectors/WKN-vectors.bin'
    ranklib_output = 'w2vwebisall'
    score_output = 'w2vwebisall.txt'
    embedding_type = 'wikipedia2vec'

    linker = '../Data/webis-qinc-22.json'
    dbpedia_input = "../src/DBpedia-Entity/runs/v2/bm25f-ca_v2.run"
    def __init__(self, i_var):
        self.i_var = i_var

path_to_dbpedia = "../src/DBpedia-Entity"

In [ ]:
if not os.path.exists(args.embedding):
    print("Embedding file does not exist! Make sure you have unzipped the file in /src")
else:
    print("Found Embeddings")
if not os.path.exists(args.dbpedia_input):
    print("Dbpedia file does not exist! Make sure you have unzipped the file in /src")
else:
    print("Found Dbpedia")
if not os.path.exists(args.linker):
    print("Linker file does not exist!")
else:
    print("Found Linker")

In [ ]:
def entity_converter(word, reverse = False, nospace = True):
    # Input: an entity string in dbpedia format
    # Output: string in wikipedia2vec format or readible format
    # Getting entities in the right format
    if args.embedding_type == 'rdf2vec':
            word = word.replace("<dbpedia:", "http://dbpedia.org/resource/")
            word = word.replace(">", "")
            return(word)
    elif args.embedding_type == 'complex':
        word = word.replace("dbpedia:", "http://dbpedia.org/resource/")
        return(word)
    elif reverse:
        word = word.replace("<dbpedia:", "")
        word = word.replace(">", "")
        if nospace:
            return word
        else:
            word = word.replace("_", " ")
            return word
    else:
        word = word.replace("<dbpedia:", "ENTITY/")
        word = word.replace(">", "")
        return word

In [ ]:
def entity_lookup(tag, model, tagme = False):
    # Input: entity string
    # Output: True wikipedia2vec format
    # Looking up the embedding of entities, returning [] when entity not in corpus
    if tagme and args.embedding_type == 'rdf2vec':
        tag = tag.replace('ENTITY/', 'http://dbpedia.org/resource/')
    elif tagme and args.embedding_type == 'complex':
        tag = tag.replace('ENTITY/', '<http://dbpedia.org/resource/')
        tag = tag + '>'
    elif tag in redirect_dict:
        tag = redirect_dict[tag]
    else:
        tag = entity_converter(tag)
    if args.embedding_type == 'wikipedia2vec': 
        if tag in model.key_to_index:
            return model[tag]
    if args.embedding_type == 'rdf2vec': 
        if tag in model.vocab:
            return model[tag]
        else:
            return []
    if args.embedding_type == 'complex': 
        #if tag in model.dataset.entity_ids():
        if tag in entity_ids_dict:
            #ent_index = model.dataset.entity_ids().index(tag)
            ent_index = entity_ids_dict[tag]
            model_tag = model.get_s_embedder().embed(torch.Tensor([ent_index]).long())
            return model_tag
        else:
            return []
    else:
        return []


In [ ]:
if args.embedding_type == 'complex':
    import torch
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
def ranking_feature(query, entity, dist = "cosine", conf = True):
    # Input: a query-entity pair to score, a distance function and if we want to use confidence scores
    # Output: query-entity based score

    # getting all the linked entities to the query
    if query not in confidence:
        return 0
    
    else:       
        max_total = []
        for a_ent in confidence[query]['interpretations']:
            total = 0
            ent_interps = [a for a in a_ent['interpretation'] if 'wikipedia' in a]
            ent_nr = 0

            for tag in ent_interps:
            #for index, row in a_ent.iterrows():
                #score = row['confidence']
                score = 1
                #score = row['score']
                if 'wikipedia' not in tag:
                    continue
                tag = 'ENTITY/' + tag.split('/')[-1]    
                ent1 = entity_lookup(entity, model)
                ent2 = entity_lookup(tag, model, tagme = True)
                #ent2 = entity_lookup(row['entity'])
                if len(ent1) == 0 or len(ent2) == 0:
                    continue
                else:
                    ent_nr += 1

                    if dist == 'fjaccard':
                        dist = 1-np.minimum(ent1,ent2).sum()/np.maximum(ent1,ent2).sum()
                    elif args.embedding_type == 'complex':
                        dist = cos(ent1, ent2).detach().numpy()[0]
                    else:
                        dist = 1 - spatial.distance.cosine(ent1, ent2)
                    if conf:
                        #score = row['confidence']
                        total += score*dist
                    else:
                        total += dist
            #total = total/ent_nr if total > 0 else 0
            #print(a_ent['interpretation'], total, ent_nr)
            max_total.append(total)
        #print(max_total, max(max_total))
        #returntotal = max(max_total)
        #returntotal = sum(max_total)/len(max_total)
        returntotal = sum(max_total)#/len(max_total)
        return returntotal

In [ ]:
def to_print_format(queries, filepath):
    # input: A list of queries of which we want the results
    # output: Ranklib ready format of results
    f = open(filepath, "w")
    for q in queries:
        entities = new_df.loc[(new_df['query_id'] == q)]
        for index, row in entities.iterrows():
            printstring = str(int(row['rel'])) + ' qid:' + row['query_id'] + " 1:" + str(row['embedding_score']) + " 2:" + str(row['fsdm_score']) + " # " + row['tag']
            print(printstring, file = f)
    f.close()

In [ ]:
def to_print_format_dict(queries, filepath):
    # input: A list of queries of which we want the results
    # output: Ranklib ready format of results
    f = open(filepath, "w")
    for q in queries:
        entities = rerank[q]
        for ent in entities.keys():
            printstring = str(int(entities[ent]['rel'])) + ' qid:' + q + " 1:" + str(entities[ent]['score']) + " 2:" + str(entities[ent]['fsdm_score']) + " # " + ent
            print(printstring, file = f)
    f.close()

In [ ]:
print("Loading auxilary files")
# Loading the file to rerank
# Note, if an error is given here when loading a different file to rerank, try changing the seperator to '\t'

qrels_path = path_to_dbpedia + '/collection/v2/qrels-v2.txt'
qrels = {}
with open(qrels_path) as f:
    for line in f:
        terms = line.split()
        if terms[0] not in qrels:
            qrels[terms[0]] = {}
        qrels[terms[0]][terms[2]] = terms[3]



rerank_path = args.dbpedia_input
rerank = {}
with open(rerank_path) as f:
    for line in f:
        terms = line.split()
        if terms[0] not in rerank:
            rerank[terms[0]] = {}
        if terms[2] in qrels[terms[0]]:
            rel = qrels[terms[0]][terms[2]]
        else:
            rel = 0
        rerank[terms[0]][terms[2]] = {'fsdm_score' : terms[4], 'rel' : rel}



# Loading auxilary files
queries_path = path_to_dbpedia + '/collection/v2/queries-v2.txt'
queries = pd.read_csv(queries_path, sep='\t',names = ['query_id', 'query'])

# Loading previously computed redirects
df = pd.read_csv('../Data/wikipedia_redirect.csv')



In [ ]:
# Getting the data ready for further processing in RankLib
folds_path = path_to_dbpedia + "/collection/v2/folds/all_queries.json"
with open(folds_path, 'r') as read_file:
    data = json.load(read_file)

In [ ]:
with open(args.linker) as json_file:
    ddata = json.load(json_file)
confidence = {}
for q in ddata['queries']:
    if 'dbpediav2' in q['id']:
        new_id = q['id'][12:]
        confidence[new_id] = q

In [ ]:
len(confidence)

In [ ]:
print("Loading embeddings")
# Loading the model with a Gensim keyedvector
if args.embedding_type == 'wikipedia2vec' or args.embedding_type == 'rdf2vec': 
    model = gensim.models.KeyedVectors.load(args.embedding, mmap='r')
elif args.embedding_type == 'complex': 
    import torch
    from kge import Dataset
    from kge.model import KgeModel
    from kge.util.io import load_checkpoint
    checkpoint = load_checkpoint(args.embedding)
    dataset = Dataset.create(checkpoint['config'],folder = '../Data/dbpedia_kge_small')
    model = KgeModel.create_from(checkpoint, dataset)
    entity_ids = model.dataset.entity_ids()
    entity_ids_dict = {}
    for i, e in enumerate(entity_ids):
        entity_ids_dict[e] = i


redirect_dict = {}
for index, tags in df.iterrows():
    redirect_dict[tags['original']] = tags['redirect']

In [ ]:
# Sanity check
entity_lookup(entity_converter('<dbpedia:Amsterdam>'), model)

In [ ]:
# Merging the qrels and rerank for scoring

# Scoring everything (might take a while)
test_x = []

f = open(args.score_output, 'w')
print("Ranking entities:")
for query_id in rerank.keys():
    print(query_id)
    for tag in rerank[query_id].keys():        
        query_based_score = ranking_feature(query_id, tag, conf = True)
        if query_based_score == 0:
            break
        test_x.append(query_based_score)
        rerank[query_id][tag]['score'] = query_based_score
        print(" ".join([query_id, tag, str(query_based_score)] ), file=f)



In [ ]:

print("Saving files for Ranklib:")
for i in range(5):
    query_fold = data[str(i)]
    folder = args.ranklib_output + "/Fold" +str(i+1)

    if not os.path.exists(folder):
        os.makedirs(folder)
    
    testpath = folder + "/test.txt"
    print("Now writing test for fold " + str(i+1))
    to_print_format_dict(query_fold['testing'], testpath)

    trainpath = folder + "/train.txt"
    print("Now writing train for fold " + str(i+1))
    test_print = to_print_format_dict(query_fold['training'], trainpath)
